In [42]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re,os,requests
import subprocess
import financedatabase as fd
from io import StringIO

import pandas as pd

In [43]:
equities = fd.Equities()
equities_united_states = equities.select(country="United States")

tickers=list(equities_united_states.query("sector=='Utilities'").index)


In [44]:
Indemt='-'
def logs(msg,indemt=Indemt[0]):
    global Indemt
    print(Indemt+msg)
    Indemt+=indemt
def end_logs(msg='done.',indemt=Indemt[0]):
    global Indemt
    Indemt=Indemt[:-1]
    print(Indemt+msg)

In [301]:

class Browser:
    browser, service = None, None
    # Initialise the webdriver with the path to chromedriver.exe
    def __init__(self, options:str ,open_on: str = ''):
        print('Launching Browser.')
        self.browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        if open_on:
            self.open_page(url=open_on)
        time.sleep(0.3)
    
    def open_page(self, url: str,new_tab=False):
        
        print(f"Opening {url}.")
        if new_tab:
            self.browser.execute_script(f'''window.open("{url}","_blank");''')
        else:
            self.browser.get(url)

    def close_browser(self):
        print('Closing Browser.')
        self.browser.close()

    def add_input(self, by: By, value: str, text: str):
        field = self.browser.find_element(by=by, value=value)
        field.send_keys(text)
        time.sleep(1)

    def click_button(self, by: By, value: str):
        button = self.browser.find_element(by=by, value=value)
        button.click()
        time.sleep(0.2)

    def login_(self, username: str, password: str):
        self.add_input(by=By.ID, value='form_login_email', text=username)
        self.add_input(by=By.NAME, value='password', text=password)
        self.click_button(
            by=By.XPATH, value="//button[@class='button login-button']")
    def ScrapTable(self,Tickers,Results=[],new_window=True):
        self.browser.switch_to.window(window_name=self.browser.window_handles[-1])
        if new_window:
            self.browser.switch_to.new_window('window')
        if isinstance(Tickers,str):
            Tickers=[Tickers]
        MC={}
        for ticker in Tickers:
            self.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")
            
            if 'Page Not Found' in self.browser.page_source:
                print('No Data found for'+ticker)
                continue
            Table=self.FetchTable()
            MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
            MC[ticker].index=pd.to_datetime(MC[ticker].index)
        Results.append(MC)
        return MC
    def FetchTable(self):
        Table=''
        tries=0
        page=1000
        while tries<10:
            try:
                pages=int(re.search(r'\d*$',self.browser.find_element(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]').text)[0])+1
                break
            except Exception:
                tries+=1
        for page in range(1,pages):
            try:
                print('fetshing data on page '+str(page))
                try:
                    self.click_button(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]')
                except NoSuchElementException:
                    time.sleep(0.3)
                    self.click_button(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]')
                try:
                    txt=self.browser.find_element(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[3]/div').text
                    if isinstance(txt,str):
                        txt=re.sub(r'(?<= \d{4}) ',";",txt.replace('Date Value\n',''))
                except (StaleElementReferenceException,NoSuchElementException):
                    time.sleep(0.3)
                    txt=re.sub(r'(?<= \d{4}) ',";",self.browser.find_element(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[3]/div').text)
                Table=f"{Table}{txt}\n"
            except ElementClickInterceptedException as e:
                break
        return Table.replace('Date Value\n','')
options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 1/")
options1 = webdriver.ChromeOptions()
options1.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 2/")


In [46]:
%store -r MC
%store -r skiped

In [47]:
len(skiped)

163

In [48]:
len(MC)

790

In [49]:
Results=[]

In [302]:
%store MC
%store skiped
Tickers="A" # list(set(equities_united_states.query("sector!='Utilities'").index)- set(MC.keys())-set(skiped)  )
browser = Browser(options,)
new_window=False
browser.browser.switch_to.window(window_name=browser.browser.window_handles[-1])
if new_window:
    browser.browser.switch_to.new_window('window')
if isinstance(Tickers,str):
    Tickers=[Tickers]
# MC={}
for ticker in Tickers:
    browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")

    if 'Page Not Found' in browser.browser.page_source:
        print('No Data found for'+ticker)
        skiped.append(ticker)
        continue
    if  "Market Cap" not in browser.browser.find_element(by=By.XPATH,value=f'/html/body/main/div/div[3]/div/div/div/div[1]/div/h1').text:
        skiped.append(ticker)
        continue
    Table=browser.FetchTable()
    if not Table:
        skiped.append(ticker)
        continue
    MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
    MC[ticker].index=pd.to_datetime(MC[ticker].index)
    MC[ticker].reset_index(inplace=True)
Results.append(MC)


Stored 'MC' (dict)
Stored 'skiped' (list)
Launching Browser.


WebDriverException: Message: unknown error: Chrome failed to start: exited normally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082DCE3+50899]
	(No symbol) [0x007BE111]
	(No symbol) [0x006C5588]
	(No symbol) [0x006E281C]
	(No symbol) [0x006DF479]
	(No symbol) [0x00711FFE]
	(No symbol) [0x00711CEC]
	(No symbol) [0x0070B6F6]
	(No symbol) [0x006E7708]
	(No symbol) [0x006E886D]
	GetHandleVerifier [0x00A93EAE+2566302]
	GetHandleVerifier [0x00AC92B1+2784417]
	GetHandleVerifier [0x00AC327C+2759788]
	GetHandleVerifier [0x008C5740+672048]
	(No symbol) [0x007C8872]
	(No symbol) [0x007C41C8]
	(No symbol) [0x007C42AB]
	(No symbol) [0x007B71B7]
	BaseThreadInitThunk [0x767D0099+25]
	RtlGetAppContainerNamedObjectPath [0x77117B6E+286]
	RtlGetAppContainerNamedObjectPath [0

In [ ]:
browser=Browser(options=options)

Launching Browser.


In [261]:
try:
    browser.click_button(by=By.XPATH,value=f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[3]')
except Exception as e:
    pass

def element_exists_by_xpath(xpath):
    try:
        browser.browser.find_element(by=By.XPATH,value=xpath)
    except NoSuchElementException:
        return False
    return True

def fetch_by_xpath(xpath):
    try:
        txt=browser.browser.find_element(by=By.XPATH,value=xpath).text
    except StaleElementReferenceException:
        time.sleep(0.05)
        txt=browser.browser.find_element(by=By.XPATH,value=xpath).text
        
    if isinstance(txt,str):
        txt=re.sub(r'(?<= \d{4}) ',";",txt.replace('Date Value\n',''))
    return txt

Table=''
tries=0
pages=30
next_button_xpath= f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]'
data_xpath='//*[@id="ycn-historical-data-table-0"]/div[3]/div'
page_number_xpath = f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]'
while tries<10:
    try:
        pages=int(re.search(r'\d*$',browser.browser.find_element(by=By.XPATH,value=page_number_xpath).text)[0])+1
        break
    except Exception:
        tries+=1

for page in range(1,pages):
    print('fetshing data on page '+str(page)+f" out of {pages-1}")
    if element_exists_by_xpath(data_xpath):
        txt_page=fetch_by_xpath(data_xpath)
        while (txt_page in Table) and page<pages-1:
            time.sleep(0.1)
            txt_page=fetch_by_xpath(data_xpath)
        
        Table=f"{Table}{txt_page}\n"
        print("copied")
        try:
            browser.click_button(by=By.XPATH,value=next_button_xpath)
        except ElementClickInterceptedException:
            pass

fetshing data on page 1 out of 29


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082DCE3+50899]
	(No symbol) [0x007BE111]
	(No symbol) [0x006C5588]
	(No symbol) [0x006AD333]
	(No symbol) [0x0070F4DB]
	(No symbol) [0x0071DB33]
	(No symbol) [0x0070B6F6]
	(No symbol) [0x006E7708]
	(No symbol) [0x006E886D]
	GetHandleVerifier [0x00A93EAE+2566302]
	GetHandleVerifier [0x00AC92B1+2784417]
	GetHandleVerifier [0x00AC327C+2759788]
	GetHandleVerifier [0x008C5740+672048]
	(No symbol) [0x007C8872]
	(No symbol) [0x007C41C8]
	(No symbol) [0x007C42AB]
	(No symbol) [0x007B71B7]
	BaseThreadInitThunk [0x767D0099+25]
	RtlGetAppContainerNamedObjectPath [0x77117B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77117B3E+238]


In [195]:

Table=''
tries=0
pages=30
next_button_xpath= f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]'
data_xpath='//*[@id="ycn-historical-data-table-0"]/div[3]/div'
page_number_xpath = f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]'
while tries<10:
    try:
        pages=int(re.search(r'\d*$',browser.browser.find_element(by=By.XPATH,value=page_number_xpath).text)[0])+1
        break
    except Exception:
        tries+=1


for page in range(1,pages):
    print('fetshing data on page '+str(page)+f" out of {pages-1}")
    if element_exists_by_xpath(data_xpath):
        txt_page=fetch_by_xpath(data_xpath)
        while (txt_page in Table) and page<pages-1:
            time.sleep(0.1)
            txt_page=fetch_by_xpath(data_xpath)
        
        Table=f"{Table}{txt_page}\n"
        print("copied")
        try:
            browser.click_button(by=By.XPATH,value=next_button_xpath)
        except ElementClickInterceptedException:
            pass
            


fetshing data on page 1 out of 197
copied
fetshing data on page 2 out of 197
copied
fetshing data on page 3 out of 197
copied
fetshing data on page 4 out of 197
copied
fetshing data on page 5 out of 197
copied
fetshing data on page 6 out of 197
copied
fetshing data on page 7 out of 197
copied
fetshing data on page 8 out of 197
copied
fetshing data on page 9 out of 197
copied
fetshing data on page 10 out of 197
copied
fetshing data on page 11 out of 197
copied
fetshing data on page 12 out of 197
copied
fetshing data on page 13 out of 197
copied
fetshing data on page 14 out of 197
copied
fetshing data on page 15 out of 197
copied
fetshing data on page 16 out of 197
copied
fetshing data on page 17 out of 197
copied
fetshing data on page 18 out of 197
copied
fetshing data on page 19 out of 197
copied
fetshing data on page 20 out of 197
copied
fetshing data on page 21 out of 197
copied
fetshing data on page 22 out of 197
copied
fetshing data on page 23 out of 197
copied
fetshing data on pag

In [187]:
element_exists_by_xpath(next_button_xpath)

True

In [178]:
df=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
df.index=pd.to_datetime(df.index)
df

,Market Cap
Date,
2023-04-24,2.616000e+03
2023-04-21,2.611000e+03
2023-04-20,2.637000e+03
2023-04-19,2.652000e+03
2023-04-18,2.634000e+03
...,...
2017-07-26,7.926600e+11
2017-07-25,7.889400e+11
2017-07-24,7.855800e+11


In [193]:
page,pages-1

(197, 197)

In [327]:

import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class ScrapeThread(threading.Thread):
    def __init__(self,List_Tickers,Results,options):
        threading.Thread.__init__(self)
        self.Results=Results
        self.options = options
        self.List_Tickers = List_Tickers
        self.skiped=[]
    
    def run(self):
        self.browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)

        self.SCRAPPING(self.List_Tickers)
        # time.wait()
    
    def open_page(self, url: str,new_tab=False):
        
        print(f"Opening {url}.")
        if new_tab:
            self.browser.execute_script(f'''window.open("{url}","_blank");''')
        else:
            self.browser.get(url)
  
    def click_button(self, by: By, value: str):
        button = self.browser.find_element(by=by, value=value)
        button.click()
    
    def is_clickable(self, by: By, value: str):
        try:
            # Wait up to 10 seconds for the element to be clickable
            element = WebDriverWait(self.browser, 5).until(EC.element_to_be_clickable((by, value)))
            return True
        except:
            return False
    
    def SCRAPPING(self,Tickers):
        MC={}
        if Tickers is None:
            Tickers=equities_united_states.query("sector=='Utilities'").index
        Tickers=list(set(Tickers)- set(self.Results.keys())-set(self.skiped))
        new_window=False
        # self.browser.switch_to.window(window_name=self.browser.window_handles[-1])
        # if new_window:
        #     self.browser.switch_to.new_window('window')
        if isinstance(Tickers,str):
            Tickers=[Tickers]
        for i,ticker in enumerate(Tickers):
            self.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")

            if 'Page Not Found' in self.browser.page_source:
                print('No Data found for'+ticker)
                self.skiped.append(ticker)
                continue
            if  "Market Cap" not in self.browser.find_element(by=By.XPATH,value=f'/html/body/main/div/div[3]/div/div/div/div[1]/div/h1').text:
                self.skiped.append(ticker)
                continue
            Table=self.FetchTable()
            if not Table:
                self.skiped.append(ticker)
                continue
            MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**12 if x.endswith("T") else float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
            MC[ticker].index=pd.to_datetime(MC[ticker].index)
            MC[ticker].reset_index(inplace=True)
            if not i%5:
                self.Results.update(MC)
            self.Results.update(MC)

    def FetchTable(self):
        
        def element_exists_by_xpath(xpath):
            try:
                self.browser.find_element(by=By.XPATH,value=xpath)
            except NoSuchElementException:
                return False
            return True
        
        def fetch_by_xpath(xpath):
            try:
                txt=self.browser.find_element(by=By.XPATH,value=xpath).text
            except StaleElementReferenceException:
                time.sleep(0.1)
                txt=self.browser.find_element(by=By.XPATH,value=xpath).text
                
            if isinstance(txt,str):
                txt=re.sub(r'(?<= \d{4}) ',";",txt.replace('Date Value\n',''))
            return txt

        Table=''
        tries=0
        pages=30
        next_button_xpath= f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]'
        data_xpath='//*[@id="ycn-historical-data-table-0"]/div[3]/div'
        page_number_xpath = f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]'
        while tries<10:
            try:
                pages=int(re.search(r'\d*$',self.browser.find_element(by=By.XPATH,value=page_number_xpath).text)[0])+1
                break
            except Exception:
                tries+=1

        for page in range(1,pages):
            print('fetshing data on page '+str(page)+f" out of {pages-1}")
            if element_exists_by_xpath(data_xpath):
                txt_page=fetch_by_xpath(data_xpath)
                while (txt_page in Table):
                    time.sleep(1)
                    txt_page=fetch_by_xpath(data_xpath)
                    
                Table=f"{Table}{txt_page}\n"
                print(len(Table))    
                # if txt_page not in Table:
                #     Table=f"{Table}{txt_page}\n"
                #     print("copied")
                try:
                    self.click_button(by=By.XPATH,value=next_button_xpath)
                    print('Clickable and clicked')
                except ElementClickInterceptedException:
                    if txt_page not in Table:
                        Table=f"{Table}{txt_page}\n"
        return Table.replace('Date Value\n','')

In [325]:
%store -r Results

In [328]:
Results={}

In [329]:
%store Results
%store -r Results
my_list = equities_united_states.query('sector=="Utilities"').index


# Determine the length of each part
part_length = len(my_list) // 4

Test_tickers = [

	my_list[:part_length],
	my_list[part_length:part_length*2],
	my_list[part_length*2:part_length*3],
	my_list[part_length*3:]

]


# Options = [webdriver.ChromeOptions()]*len(Test_tickers)

options2 = webdriver.ChromeOptions()
options2.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 3/")
options3 = webdriver.ChromeOptions()
options3.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 4/")


Options = [options,options1,options2,options3]

threads = []
for i,Tickers in enumerate(Test_tickers):
	# Options[i].add_argument(f"--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile {i+1}/")
	t = ScrapeThread(Tickers,Results,Options[i])
	t.start()
	threads.append(t)

for t in threads:
	t.join()	


Stored 'Results' (dict)
Opening https://ycharts.com/companies/NFE/market_cap.
Opening https://ycharts.com/companies/APRDO/market_cap.
Opening https://ycharts.com/companies/XGEND/market_cap.
Opening https://ycharts.com/companies/DUK/market_cap.
fetshing data on page 1 out of 22
1136
Clickable and clicked
fetshing data on page 2 out of 22
No Data found forXGEND
Opening https://ycharts.com/companies/SJIV/market_cap.
fetshing data on page 1 out of 197
1136
Clickable and clicked
fetshing data on page 2 out of 197
2366
Clickable and clicked
fetshing data on page 3 out of 22
2366
Clickable and clicked
fetshing data on page 3 out of 197
No Data found forSJIV
Opening https://ycharts.com/companies/VST/market_cap.
3605
Clickable and clicked
fetshing data on page 4 out of 22
3605
Clickable and clicked
fetshing data on page 4 out of 197
4756
Clickable and clicked
fetshing data on page 5 out of 22
4756
Clickable and clicked
fetshing data on page 5 out of 197
5789
Clickable and clicked
fetshing data 

In [331]:
len(Results.keys())

90

In [333]:

cc_MC=dict(zip(Results.keys(),list(map(lambda x: x.set_index("Date") ,Results.values()))))

In [335]:
pd.concat(cc_MC,axis=1).to_csv(r'Data\Scraped MarketCap\Utilities.csv')


In [260]:
Browser(options,)


Launching Browser.


In [64]:
Browser(options1,)

Launching Browser.


In [34]:
%store MC
corrected_MC=dict(zip(MC.keys(),list(map(lambda x: x[['Date','Market Cap']],MC.values()))))
pd.concat(MC,axis=1).to_csv('Data/Scraped MarketCap/NON-Utilities.csv')

Stored 'MC' (dict)


In [105]:

Tickers=tickers
new_window=True
browser.browser.switch_to.window(window_name=browser.browser.window_handles[-1])
if new_window:
    browser.browser.switch_to.new_window('window')
if isinstance(Tickers,str):
    Tickers=[Tickers]
MC={}
for ticker in Tickers:
    browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")
    
    if 'Page Not Found' in browser.browser.page_source:
        logs('No Data found for'+ticker)
        end_logs()
        continue
    Table=browser.FetchTable()
    if not Table:
        continue
    MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
    MC[ticker].index=pd.to_datetime(MC[ticker].index)
Results.append(MC)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082DCE3+50899]
	(No symbol) [0x007BE111]
	(No symbol) [0x006C5588]
	(No symbol) [0x006B70CB]
	(No symbol) [0x006B778A]
	(No symbol) [0x006B7728]
	(No symbol) [0x006AC37A]
	(No symbol) [0x006ACBD7]
	(No symbol) [0x007144AB]
	(No symbol) [0x0070B8C3]
	(No symbol) [0x006E7708]
	(No symbol) [0x006E886D]
	GetHandleVerifier [0x00A93EAE+2566302]
	GetHandleVerifier [0x00AC92B1+2784417]
	GetHandleVerifier [0x00AC327C+2759788]
	GetHandleVerifier [0x008C5740+672048]
	(No symbol) [0x007C8872]
	(No symbol) [0x007C41C8]
	(No symbol) [0x007C42AB]
	(No symbol) [0x007B71B7]
	BaseThreadInitThunk [0x767D0099+25]
	RtlGetAppContainerNamedObjectPath [0x77117B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77117B3E+238]


In [42]:
from functools import reduce

reduce(lambda  left,right: pd.merge(left,right,on=['DATE'],
                                            how='outer'), MC.values())

KeyError: 'DATE'

In [ ]:
browser.browser.find_element(By.TAG_NAME ,'body').send_keys(Keys.COMMAND + 't') 

In [ ]:
window_name = browser.browser.window_handles[-1]

In [ ]:
browser.browser.switch_to.window(window_name=window_name)

In [ ]:
browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")